In [1]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("train.csv")
data = Dataset.from_pandas(df)

/mnt/c/Users/ankit/Desktop/Portfolio/kaggle/kaggle-experiments/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv

load_dotenv("/mnt/c/Users/ankit/Desktop/Portfolio/kaggle/kaggle-experiments/.env")

True

In [4]:
SYSTEM_PROMPT = """**You are an advanced data generation assistant.**  
Your goal is to generate high-quality synthetic data samples based on 
provided examples. Your output must be well-structured, 
logically sound, and formatted correctly. 

**Instructions:**
1. **Follow the Structure**  
   Each data sample must include:  
   - **Description**: A clear, well-formed description of an SVG image.  
   - **Rationale**: A step-by-step, reasoning process for generation of the Description.  

2. **Output Format (Strict)**
   - A list of json consisteing of five data samples
```
[{"description" : "Generated description"
"rationale": "reasoning why this description makes sense as a data sample"},
...
]
```

**Now, generate 5 new data samples based on the given examples.**
"""

In [6]:
import re
import random

def process_response(content):
    content = content.split("```json")[1]
    content = content.split("```")[0]
    return content.strip()


def process_reresponse(content):
    pattern = r"```json(.*?)```"
    matches = re.findall(pattern, content)
    if matches:
        return matches[0].strip()
    else:
        return content

## 500 high quality samples

In [7]:
import os
import time
from json import loads

from llm import llm

model = llm(model="gemini-2.5-flash-preview-04-17", api_key=os.getenv("GEMINI_API_KEY"))

model.messages.append({"role": "system", "content": SYSTEM_PROMPT})

total_samples = 500
num_samples_gen = 5
sample_collection = []

for i in range(int(total_samples / num_samples_gen)):
    indices = [random.randrange(0, len(data)) for _ in range(num_samples_gen)]

    USER_PROMPT = "<examples>" + "\n".join(data[indices]["description"]) + "</examples>"

    response = model(USER_PROMPT)

    try:
        samples = loads(process_response(response))
    except Exception:
        samples = [{"description": ""}] * num_samples_gen

    for sample in samples:
        sample_collection.append(sample["description"])

    model.messages.pop()
    time.sleep(2)

In [ ]:
unique_sample_collection = set(sample_collection)
dict_samples = {}
dict_samples["description"] = list(unique_sample_collection)

df_save = pd.DataFrame(dict_samples)

TypeError: 'set' type is unordered

In [ ]:
dict_samples = {'description': list(dict_samples['description'])}


In [12]:
dict_samples
df_save = pd.DataFrame(dict_samples)

In [13]:
df_save.to_csv("new_train.csv")